In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
train = pd.read_csv('/kaggle/input/training-data/TrainingWiDS2021.csv')

In [3]:
test = pd.read_csv('/kaggle/input/testdataset/UnlabeledWiDS2021.csv')

In [ ]:
'''from matplotlib import pyplot
names = ['ethnicity', 'hospital_admit_source', 'icu_admit_source', 'icu_stay_type', 'icu_type', 'diabetes_mellitus']
correlations = train.corr()
fig = pyplot.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(correlations, vmin=-1, vmax=1)
fig.colorbar(cax)
#ticks = np.arange(0,9,1)
#ax.set_xticks(ticks)
#ax.set_yticks(ticks)
#ax.set_xticklabels(names)
#ax.set_yticklabels(names)
pyplot.show()'''

**Data Preprocessing for Train data**

In [4]:
train.shape

In [5]:
pd.set_option('display.max_rows', None)  
pd.set_option('display.max_columns', None)    
pd.set_option('max_colwidth', None) 

In [6]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [7]:
print(train.columns.tolist())

In [8]:
train.columns

In [9]:
train.drop("Unnamed: 0", axis = 1, inplace = True)

In [10]:
train.isnull().sum()

In [11]:
train.isnull().sum().sum()

In [12]:
# finding those columns with >70% null values
train.columns[train.isnull().mean() > .70].tolist()

In [13]:
# dropping the columns with >70% null values
for col in train:
    if train[col].isnull().mean() > .70:
        train=train.drop(col, axis = 1)

In [14]:
train.shape

In [15]:
# import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


# run correlation matrix and plot
f, ax = plt.subplots(figsize=(30, 28))
corr = train.corr()
sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool),cmap=sns.diverging_palette(220, 10, as_cmap=True),square=True, ax=ax)

In [16]:
# dropping the columns which have no relation with the independent variable
cols_drop = ['encounter_id', 'hospital_id', 'elective_surgery','ethnicity','hospital_admit_source','icu_admit_source',
             'icu_id', 'icu_stay_type', 'icu_type', 'pre_icu_los_days','readmission_status']
train = train.drop(cols_drop, axis=1)
train.shape

In [17]:
train.dtypes

In [18]:
train.dtypes.value_counts()

In [19]:
train['gender'].value_counts(dropna = False)

In [20]:
# Missing value for Categorical variables
train['gender'].fillna(value = 'M', inplace = True)

In [21]:
train['gender'].value_counts(dropna = False)

In [22]:
# The category codes of the categorical data. Codes are an array of integers which are the positions of the actual 
# values in the categorical column.

train['gender'] = pd.Categorical(train['gender']).codes

In [23]:
# replacing null values in the Numerical data columns with the mean of their values
for col in train.columns:
    train[col].fillna(train[col].mean(), inplace=True)

In [24]:
train.isnull().sum()

In [25]:
#train['age'].fillna(train['age'].mean(), inplace = True)

In [26]:
train.isnull().sum().sum()

In [27]:
train.dtypes.value_counts()

**Data Preprocessing for Test Data**

In [28]:
test.shape

In [29]:
test.columns

In [30]:
test.drop("Unnamed: 0", axis = 1, inplace = True)

In [31]:
test.isnull().sum()

In [32]:
test.isnull().sum().sum()

In [33]:
# finding the columns which have more than 70% Null values
test.columns[test.isnull().mean() > .70].tolist()

In [34]:
# dropping the columns having >70% null values
for col in test:
    if test[col].isnull().mean() > .70:
        test=test.drop(col, axis = 1)

In [35]:
test.shape

In [36]:
# dropping the columns which have no relation with the dependent variable

cols_drop = ['encounter_id', 'hospital_id', 'elective_surgery','ethnicity','hospital_admit_source','icu_admit_source',
             'icu_id', 'icu_stay_type', 'icu_type', 'pre_icu_los_days','readmission_status']
test = test.drop(cols_drop, axis=1)
test.shape

In [37]:
test.dtypes

In [38]:
test.dtypes.value_counts()

In [39]:
# replacing the null values in the categorical data column
test['gender'].value_counts(dropna = False)

In [40]:
test['gender'].fillna(value = 'M', inplace = True)

In [41]:
test['gender'].value_counts(dropna = False)

In [42]:
# The category codes of the categorical data. Codes are an array of integers which are the positions of the actual 
# values in the categorical column.

test['gender'] = pd.Categorical(test['gender']).codes

In [43]:
# replacing all the null values in the Numerical data columns with the mean of values occuring in that column
for col in test.columns:
    test[col].fillna(test[col].mean(), inplace=True)

In [44]:
test.isnull().sum()

**Logistic Regression for 70-30 dataset**

In [45]:
# Extracting the dependent and independent variables
x = train.drop("diabetes_mellitus" , axis=1)
y = train.pop("diabetes_mellitus")

In [46]:
# Splitting the train dataset into testing and training set
from sklearn.model_selection import train_test_split
xTrain,xTest,yTrain,yTest = train_test_split(x,y,test_size = 0.30,random_state = 0)

In [47]:
# Feature Scaling  
from sklearn.preprocessing import StandardScaler    
st_x= StandardScaler()    
xTrain= st_x.fit_transform(xTrain)    
xTest= st_x.transform(xTest)  

In [48]:
# Fitting Logistic Regression to the training set
from sklearn.linear_model import LogisticRegression  
classifier= LogisticRegression(random_state=0)  
classifier.fit(xTrain, yTrain)  

In [49]:
# Predicting the test set result  
yPred= classifier.predict(xTest)  

In [50]:
# Creating the Confusion matrix to test the accuracy 
from sklearn.metrics import confusion_matrix  
cm= confusion_matrix(yTest,yPred)  

In [51]:
cm

In [52]:
# 29201+2585 = 31786 -> Correct Outputs
#1387+5875 = 7262 -> Incorrect Outputs

In [53]:
from sklearn.metrics import accuracy_score
print(accuracy_score(yTest,yPred))

**Decision Tree**

In [54]:
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

In [55]:
model = tree.DecisionTreeClassifier()
model = model.fit(xTrain, yTrain)

In [56]:
predicted_value = model.predict(xTest)

In [57]:
print("ACCURACY OF THE MODEL: ",accuracy_score(yTest, predicted_value)*100)

In [58]:
predicted_value = model.predict(xTrain)

In [59]:
print("ACCURACY OF THE MODEL: ",accuracy_score(yTrain, predicted_value)*100)

**Random Forest**

In [60]:
clf = RandomForestClassifier()
clf.fit(xTrain, yTrain)
yPred = clf.predict(xTest)
print("ACCURACY OF THE MODEL: ",accuracy_score(yTest, yPred)*100)

In [61]:
y_pred = clf.predict(xTrain)
print("ACCURACY OF THE MODEL: ",accuracy_score(yTrain, y_pred)*100)

**XG Boost**

In [62]:
from xgboost import XGBClassifier

In [63]:
model = XGBClassifier()

In [64]:
model.fit(xTrain, yTrain)

In [65]:
y_pred = model.predict(xTest)

In [66]:
accuracy_score(yTest, y_pred)

**Deployment**

In [67]:
#pip install gradio

In [68]:
import gradio as gr

In [69]:
train.shape

In [70]:
print(train.columns)

In [71]:
#train.drop('diabetes_mellitus',inplace = True)

In [72]:
train.index.name = "index"

In [73]:
index = [x for x in range(1,130158)]

In [74]:
train.index = index

In [76]:
import pickle

In [77]:
pickle.dump(clf,open('csv.sav','wb'))

In [78]:
model = pickle.load(open('csv.sav','rb'))

In [79]:
def predict_res(id):
    x = train.loc[id].to_numpy()
    a_stand = (x - xTrain.mean())/(xTrain.std())
    if model.predict([a_stand]) == 0:
        return "The patient is not diagnosed with Diabetes Mellitus"
    else :
        return "The patient is  diagnosed with Diabetes Mellitus"

In [80]:
title = "Prediction of Diabetes Mellitus"
description = """
<center>
The bot is trained to predict whether a patient admitted to an ICU has been diagnosed with a particular type of diabetes, Diabetes Mellitus.
<img src="https://m.economictimes.com/thumb/msid-61997607,width-1200,height-900,resizemode-4,imgsize-175312/suffering-from-diabetes-five-apps-to-help-you-manage-your-lifestyle-better.jpg" width="300" height="200">
Give the ID of the patient
</center>
"""

In [81]:
outputs = gr.outputs.Textbox()

In [82]:
gr.Interface(predict_res, inputs="number", outputs=outputs,title = title,description = description).launch(share = True)